### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Osteoporosis/GSE20881'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
if 'gene expression' in sample_characteristics_dict['!Series_summary']:
    is_gene_available = True

# Data availability check and identifying keys
for key, values in sample_characteristics_dict.items():
    if any('Osteoporosis' in str(x) for x in values if isinstance(x, str)):
        trait_row = key

    if any('birth date' in str(x) for x in values if isinstance(x, str)):
        age_row = key
    
    # Assuming gender data might be referred to with other inferred keys, not directly present
    if any('gender' in str(x) for x in values if isinstance(x, str)):
        gender_row = key  # Assume gender identifier is present

# Convert trait function
def convert_trait(value):
    if 'Osteoporosis' in value:
        return 1
    elif 'none' in value:
        return 0
    return None

# Convert age function
from datetime import datetime

def convert_age(value):
    try:
        birth_date_str = value.split(":")[1].strip()
        birth_date = datetime.strptime(birth_date_str, '%m/%d/%y')
        age = (datetime.now() - birth_date).days // 365
        return age
    except ValueError:
        try:
            birth_date = datetime.strptime(birth_date_str, '%Y')
            age = (datetime.now() - birth_date).days // 365
            return age
        except:
            return None

# Convert gender function
def convert_gender(value):
    if 'Male' in value:
        return 1
    elif 'Female' in value:
        return 0
    return None

# Save cohort information
save_cohort_info('GSE20881', './preprocessed/Osteoporosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction, if applicable
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Osteoporosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Osteoporosis/trait_data/GSE20881.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
